In [7]:
# !pip install segmentation-models-pytorch
# !pip install pytorch_msssim
# !pip install lightning
# !pip install lightning[extra]
# !pip install tensorboard
# !pip install scikit-image

In [2]:
# !nvcc --version

In [3]:
!pip install torch pillow torchvision opencv-python numpy

  Using cached opencv_python-4.10.0.84-cp37-abi3-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (20 kB)
  Using cached filelock-3.16.1-py3-none-any.whl.metadata (2.9 kB)
  Using cached fsspec-2024.10.0-py3-none-any.whl.metadata (11 kB)
  Using cached nvidia_cuda_nvrtc_cu12-12.4.127-py3-none-manylinux2014_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cuda_runtime_cu12-12.4.127-py3-none-manylinux2014_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cuda_cupti_cu12-12.4.127-py3-none-manylinux2014_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cudnn_cu12-9.1.0.70-py3-none-manylinux2014_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cublas_cu12-12.4.5.8-py3-none-manylinux2014_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cufft_cu12-11.2.1.3-py3-none-manylinux2014_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_curand_cu12-10.3.5.147-py3-none-manylinux2014_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cusolver_cu12-11.6.1.9-py3-none-manylinux2014_x86_64.w

In [1]:
import random
import numpy as np
import os

import torch
from PIL import Image
import torchvision.transforms as transforms
from torch.utils.data import Dataset, DataLoader
import torch.nn as nn
import torch.optim as optim
import cv2

import zipfile

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Using device:", device)

Using device: cuda


In [2]:
print(torch.cuda.is_available())  # True인지 확인
print(torch.cuda.device_count())  # 사용 가능한 GPU 개수 확인
print(torch.cuda.current_device())  # 현재 사용 중인 GPU 인덱스 확인
print(torch.cuda.get_device_name(0))  # 사용 중인 GPU 이름 확인
print(torch.version.cuda)  # PyTorch가 사용하는 CUDA 버전 출력

True
1
0
NVIDIA GeForce RTX 4070 Laptop GPU
12.4


In [3]:
origin_dir = 'train_gt'
damage_dir = 'train_input'

print(len(os.listdir(origin_dir)))
print(len(os.listdir(damage_dir)))

29603
29603


In [4]:
CFG = {
    'EPOCHS':2,
    'LEARNING_RATE':3e-4,
    # 'BATCH_SIZE':16,
    'BATCH_SIZE':16,
    'SEED':42
}

In [5]:
def seed_everything(seed):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True  # 결정론적 연산 보장
    torch.backends.cudnn.benchmark = False     # 성능 최적화 대신 일관성 우선

seed_everything(CFG['SEED'])  # Seed 고정

In [6]:
seed_everything(42)

# 테스트 난수
print("Random:", random.random())
print("NumPy Random:", np.random.rand(1))
print("PyTorch Random:", torch.rand(1))

Random: 0.6394267984578837
NumPy Random: [0.37454012]
PyTorch Random: tensor([0.8823])


In [15]:
def get_input_image(damage_img_path, origin_img_path):
    # OpenCV로 이미지 읽기 (NumPy 배열로 읽음)
    color_image = cv2.imread(origin_img_path)
    gray_image = cv2.imread(damage_img_path, cv2.IMREAD_GRAYSCALE)  # 흑백 이미지로 읽기
    
    # 색상 이미지를 흑백으로 변환 (PIL로 변환 후 NumPy로 변환)
    color_image_gray = cv2.cvtColor(color_image, cv2.COLOR_BGR2GRAY)
    
    # 두 이미지의 차이 계산
    difference = cv2.absdiff(color_image_gray, gray_image)
    
    # 차이 값을 임계값으로 처리하여 이진화 이미지 생성
    _, binary_difference = cv2.threshold(difference, 1, 255, cv2.THRESH_BINARY)

    # 마스크 생성
    mask = binary_difference > 0  # 차이가 있는 부분을 마스크로 설정
    mask = Image.fromarray(mask.astype(np.uint8) * 255)  # 마스크 이미지를 PIL 형식으로 변환

    return {
        'image_gray_masked': Image.fromarray(gray_image),  # 손상된 이미지를 PIL 이미지로 반환
        'mask': transforms.ToTensor()(mask)  # 마스크를 텐서로 변환하여 사용
    }

In [21]:
class CustomDataset(Dataset):
    def __init__(self, damage_dir, origin_dir, transform=None, use_masks=True):
        self.damage_dir = damage_dir
        self.origin_dir = origin_dir
        self.transform = transform
        self.use_masks = use_masks
        self.damage_files = sorted(os.listdir(damage_dir), key=lambda x: x.lower())
        self.origin_files = sorted(os.listdir(origin_dir), key=lambda x: x.lower())

    def __len__(self):
        return len(self.damage_files)

    def __getitem__(self, idx):
        damage_img_name = self.damage_files[idx]
        origin_img_name = self.origin_files[idx]

        damage_img_path = os.path.join(self.damage_dir, damage_img_name)
        origin_img_path = os.path.join(self.origin_dir, origin_img_name)

        damage_img = Image.open(damage_img_path).convert("RGB")
        origin_img = Image.open(origin_img_path).convert("RGB")

        if self.use_masks:
            input_data = get_input_image(damage_img_path, origin_img_path)
            mask = input_data['mask']
            # `mask`가 이미 텐서인지 확인하고 변환 처리
            if not isinstance(mask, torch.Tensor):
                mask = transforms.ToTensor()(mask)
        else:
            mask = torch.zeros((1, damage_img.size[1], damage_img.size[0]))

        if self.transform:
            damage_img = self.transform(damage_img)
            origin_img = self.transform(origin_img)

        return {'A': damage_img, 'B': origin_img, 'mask': mask}


In [22]:
# todo:
# 1. val_score
# 2. get_input_image(image) 테스트해서 원본 (손상된 흑백)과 마스크(512*512로 나오되 배경은 다 0, 마스크부분은 회색)
# 3. loss 잘못됨 (아마 컬러를 흑백으로 바꾸면 걔를 정답으로 써도 될듯)

In [1]:
# training_step의 loss계산부분 변경 -> 원본 컬러사진을 흑백으로 변환해서 비고하는 것으로
# validation_step의 score 반환부분 변경 -> 학습완료시 최적의 모델 저장이 되지않는 부분으로 실제 학습에 적용이 되는지 확인필요

In [25]:
from segmentation_models_pytorch import UnetPlusPlus
import torch.nn.functional as F
import lightning as L
from skimage.metrics import structural_similarity as ssim
import cv2
import numpy as np
import torch

def get_histogram_similarity(true_np, pred_np, color_space=cv2.COLOR_RGB2HSV):
    # BGR 이미지를 HSV로 변환
    true_hsv = cv2.cvtColor(true_np.astype(np.uint8), color_space)
    pred_hsv = cv2.cvtColor(pred_np.astype(np.uint8), color_space)
    
    # 히스토그램 계산 및 비교
    hist_true = cv2.calcHist([true_hsv], [0], None, [180], [0, 180])
    hist_pred = cv2.calcHist([pred_hsv], [0], None, [180], [0, 180])
    
    hist_true = cv2.normalize(hist_true, hist_true).flatten()
    hist_pred = cv2.normalize(hist_pred, hist_pred).flatten()
    
    similarity = cv2.compareHist(hist_true, hist_pred, cv2.HISTCMP_CORREL)
    return similarity

def get_masked_ssim_score(true_np, pred_np, mask_np):
    # true_np: (height, width, channels)
    # pred_np: (height, width, channels)
    # mask_np: (height, width) or (1, height, width)
    
    # mask_np 차원 변환
    if mask_np.ndim == 3 and mask_np.shape[0] == 1:
        mask_np = mask_np.squeeze(0)  # (1, height, width) -> (height, width)
    elif mask_np.ndim == 3 and mask_np.shape[-1] == 1:
        mask_np = mask_np.squeeze(-1)  # (height, width, 1) -> (height, width)
    
    # true_np와 mask_np의 크기가 맞지 않으면 리사이즈
    if true_np.shape[:2] != mask_np.shape:
        mask_np = cv2.resize(mask_np, (true_np.shape[1], true_np.shape[0]), interpolation=cv2.INTER_NEAREST)
    
    # 마스크가 적용된 부분만 추출
    true_masked = true_np[mask_np > 0]
    pred_masked = pred_np[mask_np > 0]

    if true_masked.size == 0 or pred_masked.size == 0:
        return 0  # 마스크된 부분이 없으면 SSIM을 0으로 처리

    # SSIM 계산
    ssim_value = ssim(
        true_masked, pred_masked, data_range=pred_masked.max() - pred_masked.min(), channel_axis=-1
    )
    return ssim_value


def ssim_score(true, pred):
    true_np = true.permute(0, 2, 3, 1).cpu().numpy()
    pred_np = pred.permute(0, 2, 3, 1).cpu().numpy()

    print(f"true_np shape: {true_np.shape}")
    print(f"pred_np shape: {pred_np.shape}")
    print(f"mask_np shape: {mask_np.shape}")

    scores = [
        ssim(t, p, channel_axis=-1, data_range=p.max() - p.min())
        for t, p in zip(true_np, pred_np)
    ]
    return np.mean(scores)

def get_ssim_score(true_np, pred_np):
    """
    두 이미지를 비교하여 SSIM(Structural Similarity Index Measure)을 계산합니다.
    NaN 값을 방지하기 위해 추가적인 조건을 추가합니다.

    Args:
        true_np (numpy.ndarray): Ground truth 이미지 (H, W, C 형태)
        pred_np (numpy.ndarray): 예측 이미지 (H, W, C 형태)

    Returns:
        float: SSIM 값 (NaN 발생 시 0으로 설정)
    """
    ssim_value = ssim(
        true_np, pred_np, channel_axis=-1, data_range=pred_np.max() - pred_np.min()
    )
    if np.isnan(ssim_value):
        ssim_value = 0  # NaN 발생 시 SSIM 값을 0으로 설정
    return ssim_value

# LitIRModel 클래스 정의
class LitIRModel(L.LightningModule):
    def __init__(self, model_1, model_2, image_mean=0.5, image_std=0.5):
        super().__init__()
        self.model_1 = model_1
        self.model_2 = model_2
        self.image_mean = image_mean
        self.image_std = image_std
        self.validation_outputs = []

    def forward(self, images_gray_masked):
        images_gray_restored = self.model_1(images_gray_masked) + images_gray_masked
        images_restored = self.model_2(images_gray_restored)
        return images_gray_restored, images_restored

    def unnormalize(self, output, round=False):
        image_restored = ((output * self.image_std + self.image_mean) * 255).clamp(0, 255)
        if round:
            image_restored = torch.round(image_restored)
        return image_restored

    def configure_optimizers(self):
        opt = torch.optim.AdamW(self.parameters(), lr=1e-5)
        return opt

    def training_step(self, batch, batch_idx):
        images_gray_masked = batch['A']  # 손상된 이미지 정답: mask가 채워진 흑백사진
        images_gt = batch['B']  # Ground Truth 이미지
        masks = batch['masks']  # 마스크 (현재 사용 여부에 따라 다름)

        # Ground Truth 이미지를 흑백으로 변환
        images_gray_masked = torch.mean(images_gt, dim=1, keepdim=True)  # RGB 평균을 계산하여 흑백으로 변환
    
        # 모델에 입력
        images_gray_restored, images_restored = self(images_gray_masked)
    
        # 손실 계산
        loss_pixel_gray = (
            F.l1_loss(images_gray_masked, images_gray_restored, reduction='mean') * 0.5 +
            F.mse_loss(images_gray_masked, images_gray_restored, reduction='mean') * 0.5
        )
        loss_pixel = (
            F.l1_loss(images_gt, images_restored, reduction='mean') * 0.5 +
            F.mse_loss(images_gt, images_restored, reduction='mean') * 0.5
        )
        loss = loss_pixel_gray * 0.5 + loss_pixel * 0.5

        # 로깅 (Batch와 손실 값 출력)
        print(f"Batch {batch_idx}, Loss: {loss.item()}")
        
        # 로그 기록
        self.log("train_loss", loss, on_step=True, on_epoch=True)
        return loss

    def validation_step(self, batch, batch_idx):
        # Extract data from batch
        images_gray_masked = batch['A']
        images_gt = batch['B']
        masks = batch['masks']

        # Forward pass
        images_gray_restored, images_restored = self(images_gray_masked)

        # Ground Truth와 복원된 이미지 크기 맞추기
        if images_restored.shape != images_gt.shape:
            images_restored = torch.nn.functional.interpolate(
                images_restored, size=images_gt.shape[-2:], mode="bilinear", align_corners=False
            )

        # NumPy 변환
        images_restored_np = images_restored.detach().cpu().permute(0, 2, 3, 1).float().numpy().astype(np.uint8)
        images_gt_np = images_gt.detach().cpu().permute(0, 2, 3, 1).float().numpy().astype(np.uint8)
        masks_np = masks.detach().cpu().numpy()

        # Metric 계산
        total_ssim_score = 0
        masked_ssim_score = 0
        hist_sim_score = 0
        for image_gt_np, image_restored_np, mask_np in zip(images_gt_np, images_restored_np, masks_np):
            total_ssim_score += get_ssim_score(image_gt_np, image_restored_np) / len(images_gt)
            masked_ssim_score += get_masked_ssim_score(image_gt_np, image_restored_np, mask_np) / len(images_gt)
            hist_sim_score += get_histogram_similarity(image_gt_np, image_restored_np) / len(images_gt)

        # 최종 점수
        score = total_ssim_score * 0.2 + masked_ssim_score * 0.4 + hist_sim_score * 0.4

        # 배치별 검증 결과 출력
        print(f"Validation Batch {batch_idx}, Total SSIM: {total_ssim_score:.4f}, "
              f"Masked SSIM: {masked_ssim_score:.4f}, Histogram Similarity: {hist_sim_score:.4f}")

        # 로깅
        self.log("val_score", score, on_step=False, on_epoch=True)
        self.log("val_total_ssim_score", total_ssim_score, on_step=False, on_epoch=True)
        self.log("val_masked_ssim_score", masked_ssim_score, on_step=False, on_epoch=True)
        self.log("val_hist_sim_score", hist_sim_score, on_step=False, on_epoch=True)

        return {'val_score': score}

    def on_validation_epoch_end(self):
        total_ssim_score = 0
        masked_ssim_score = 0
        hist_sim_score = 0

        for output in self.validation_outputs:
            images_gt = output["images_gt"]
            images_restored = output["images_restored"]
            masks = output["masks"]

            masks_np = masks.detach().cpu().numpy()
            images_gt_np = images_gt.detach().cpu().permute(0, 2, 3, 1).float().numpy().astype(np.uint8)
            images_restored_np = images_restored.detach().cpu().permute(0, 2, 3, 1).float().numpy().astype(np.uint8)

            for image_gt_np, image_restored_np, mask_np in zip(images_gt_np, images_restored_np, masks_np):
                total_ssim_score += get_ssim_score(image_gt_np, image_restored_np) / len(images_gt_np)
                masked_ssim_score += get_masked_ssim_score(image_gt_np, image_restored_np, mask_np) / len(images_gt_np)
                hist_sim_score += get_histogram_similarity(image_gt_np, image_restored_np, cv2.COLOR_RGB2HSV) / len(images_gt_np)

        score = total_ssim_score * 0.2 + masked_ssim_score * 0.4 + hist_sim_score * 0.4

        self.log(f"val_score", score, on_step=False, on_epoch=True)
        self.log(f"val_total_ssim_score", total_ssim_score, on_step=False, on_epoch=True)
        self.log(f"val_masked_ssim_score", masked_ssim_score, on_step=False, on_epoch=True)
        self.log(f"val_hist_sim_score", hist_sim_score, on_step=False, on_epoch=True)

        self.validation_outputs = []

# 모델 초기화
# 첫 번째 모델: Gray Mask Restoration
model_1 = UnetPlusPlus(
    encoder_name="efficientnet-b4",
    encoder_weights="imagenet",
    in_channels=3,
    classes=1
)

# 두 번째 모델: Gray → Color
model_2 = UnetPlusPlus(
    encoder_name="efficientnet-b4",
    encoder_weights="imagenet",
    in_channels=3,
    classes=3
)

# LitIRModel 초기화
lit_ir_model = LitIRModel(model_1=model_1, model_2=model_2)


In [26]:
from torch.utils.data import random_split, DataLoader
import torchvision.transforms as transforms
from lightning.pytorch.callbacks import ModelCheckpoint, EarlyStopping
import lightning as L
import os
import torch

# 경로 설정
origin_dir = 'train_gt'
damage_dir = 'train_input'
test_dir = 'test_input'

# 데이터 전처리 설정
transform = transforms.Compose([
    transforms.Resize((256, 256)),  # 모든 이미지를 256x256으로 리사이즈
    transforms.ToTensor(),
    transforms.Normalize([0.5], [0.5])
])

# 데이터셋 생성
dataset = CustomDataset(damage_dir=damage_dir, origin_dir=origin_dir, transform=transform)

# 데이터셋 분할
validation_ratio = 0.2
train_size = int((1 - validation_ratio) * len(dataset))
val_size = len(dataset) - train_size
training_dataset, validation_dataset = random_split(dataset, [train_size, val_size])

class CollateFn:
    def __init__(self, mode='train'):
        self.mode = mode

    def __call__(self, batch):
        A = torch.stack([item['A'] for item in batch])
        B = torch.stack([item['B'] for item in batch])
        masks = torch.stack([item['mask'] for item in batch]) if 'mask' in batch[0] else torch.zeros_like(A)

        if self.mode in ['train', 'valid']:
            return {'A': A, 'B': B, 'masks': masks}
        elif self.mode == 'test':
            return {'A': A}

# CollateFn 정의
train_collate_fn = CollateFn(mode='train')
validation_collate_fn = CollateFn(mode='valid')

# DataLoader 설정
train_dataloader = DataLoader(
    training_dataset,
    batch_size=CFG['BATCH_SIZE'],
    shuffle=True,
    num_workers=8,
    pin_memory=True,
    collate_fn=train_collate_fn
)

validation_dataloader = DataLoader(
    validation_dataset,
    batch_size=CFG['BATCH_SIZE'],
    shuffle=False,
    num_workers=8,
    collate_fn=validation_collate_fn
)

# 모델 저장 디렉토리 생성
model_save_dir = "./saved_models"
os.makedirs(model_save_dir, exist_ok=True)

# Trainer 설정 및 학습 시작
trainer = L.Trainer(
    max_epochs=CFG['EPOCHS'],
    precision=16,
    accelerator='gpu',
    devices=1,
    log_every_n_steps=10,
    callbacks=[
        ModelCheckpoint(
            monitor='val_score',
            mode='max',
            save_top_k=10 ,
            dirpath=model_save_dir,
            filename='best_model-{epoch:02d}-{val_score:.4f}'
        ),
        EarlyStopping(monitor='val_score', mode='max', patience=3)
    ]
)

# 학습 시작
trainer.fit(lit_ir_model, train_dataloader, validation_dataloader)

Using 16bit Automatic Mixed Precision (AMP)
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name    | Type         | Params | Mode 
-------------------------------------------------
0 | model_1 | UnetPlusPlus | 20.8 M | train
1 | model_2 | UnetPlusPlus | 20.8 M | train
-------------------------------------------------
41.6 M    Trainable params
0         Non-trainable params
41.6 M    Total params
166.506   Total estimated model params size (MB)
1274      Modules in train mode
0         Modules in eval mode


Sanity Checking: |                                                                                | 0/? [00:00…

Validation Batch 0, SSIM Score: 0.0442
Validation Batch 1, SSIM Score: 0.0418


Training: |                                                                                       | 0/? [00:00…

Batch 0, Loss: 0.7354874610900879
Batch 1, Loss: 0.7647855877876282
Batch 2, Loss: 0.7115100622177124
Batch 3, Loss: 0.7242957353591919
Batch 4, Loss: 0.7083770632743835
Batch 5, Loss: 0.7217937111854553
Batch 6, Loss: 0.7273175716400146
Batch 7, Loss: 0.7070827484130859
Batch 8, Loss: 0.6934892535209656
Batch 9, Loss: 0.7339079976081848
Batch 10, Loss: 0.7161211967468262
Batch 11, Loss: 0.6992403268814087
Batch 12, Loss: 0.7116003036499023
Batch 13, Loss: 0.7213436961174011
Batch 14, Loss: 0.7130628824234009
Batch 15, Loss: 0.6493666172027588
Batch 16, Loss: 0.695054292678833
Batch 17, Loss: 0.700727105140686
Batch 18, Loss: 0.6940500736236572
Batch 19, Loss: 0.716075599193573
Batch 20, Loss: 0.6800341606140137
Batch 21, Loss: 0.7064216732978821
Batch 22, Loss: 0.718167245388031
Batch 23, Loss: 0.6697914600372314
Batch 24, Loss: 0.6721053719520569
Batch 25, Loss: 0.6921496391296387
Batch 26, Loss: 0.6521962881088257
Batch 27, Loss: 0.6533606052398682
Batch 28, Loss: 0.6893563270568848

...

Validation Batch 0, SSIM Score: 0.9238
Validation Batch 1, SSIM Score: 0.9866
Validation Batch 2, SSIM Score: 0.9152
Validation Batch 3, SSIM Score: 0.8943
Validation Batch 4, SSIM Score: 0.9179
Validation Batch 5, SSIM Score: 0.9565
Validation Batch 6, SSIM Score: 0.9034
Validation Batch 7, SSIM Score: 0.8910
Validation Batch 8, SSIM Score: 0.9437
Validation Batch 9, SSIM Score: 0.7955
Validation Batch 10, SSIM Score: 0.9096
Validation Batch 11, SSIM Score: 0.9266
Validation Batch 12, SSIM Score: 0.9697
Validation Batch 13, SSIM Score: 0.9006
Validation Batch 14, SSIM Score: 0.9501
Validation Batch 15, SSIM Score: 0.9175
Validation Batch 16, SSIM Score: 0.8898
Validation Batch 17, SSIM Score: 0.8561
Validation Batch 18, SSIM Score: 0.8969
Validation Batch 19, SSIM Score: 0.9002
Validation Batch 20, SSIM Score: 0.8951
Validation Batch 21, SSIM Score: 0.9680
Validation Batch 22, SSIM Score: 0.8906
Validation Batch 23, SSIM Score: 0.9681
Validation Batch 24, SSIM Score: 0.8974
Validation

Validation: |                                                                                     | 0/? [00:00…

Validation Batch 0, SSIM Score: 0.9863
Validation Batch 1, SSIM Score: 0.9867
Validation Batch 2, SSIM Score: 0.9128
Validation Batch 3, SSIM Score: 0.9564
Validation Batch 4, SSIM Score: 0.9189
Validation Batch 5, SSIM Score: 0.8947
Validation Batch 6, SSIM Score: 0.8451
Validation Batch 7, SSIM Score: 0.9569
Validation Batch 8, SSIM Score: 0.8844
Validation Batch 9, SSIM Score: 0.8608
Validation Batch 10, SSIM Score: 0.9125
Validation Batch 11, SSIM Score: 0.8627
Validation Batch 12, SSIM Score: 0.9659
Validation Batch 13, SSIM Score: 0.9016
Validation Batch 14, SSIM Score: 0.9492
Validation Batch 15, SSIM Score: 0.9172
Validation Batch 16, SSIM Score: 0.8941
Validation Batch 17, SSIM Score: 0.9194
Validation Batch 18, SSIM Score: 0.9452
Validation Batch 19, SSIM Score: 0.8935
Validation Batch 20, SSIM Score: 0.8368
Validation Batch 21, SSIM Score: 0.9100
Validation Batch 22, SSIM Score: 0.8870
Validation Batch 23, SSIM Score: 0.9721
Validation Batch 24, SSIM Score: 0.8875
Validation

`Trainer.fit` stopped: `max_epochs=2` reached.


Validation Batch 369, SSIM Score: 0.9092
Validation Batch 370, SSIM Score: 0.9997


In [ ]:
from torch.utils.data import DataLoader
import os
from PIL import Image
import torch
import torchvision.transforms as transforms
import matplotlib.pyplot as plt

# 데이터 전처리
transform = transforms.Compose([
    transforms.Resize((256, 256)),
    transforms.ToTensor(),
    transforms.Normalize([0.5], [0.5])
])


# 테스트 데이터셋 생성
test_dataset = CustomDataset(damage_dir=test_dir, origin_dir=test_dir, transform=transform, use_masks=False)

# 테스트 DataLoader 생성
test_dataloader = DataLoader(
    test_dataset,
    batch_size=CFG['BATCH_SIZE'],  # 적절히 설정
    shuffle=False,
    num_workers=2  # 경고를 피하기 위해 적절히 설정
)

# 모델 초기화
model = LitIRModel.load_from_checkpoint(
    checkpoint_path='saved_models/best_model-epoch=01-val_score=0.0000.ckpt',
    model_1=model_1,
    model_2=model_2
)

# 모델을 평가 모드로 설정
model.eval()
model.to(device)

# 테스트 데이터로 예측 실행
output_dir = "output/"
os.makedirs(output_dir, exist_ok=True)

model.eval()  # 모델을 평가 모드로 설정
with torch.no_grad():
    for idx, batch in enumerate(test_dataloader):
        # 입력 데이터 준비 (RGB -> Grayscale 변환)
        inputs = torch.mean(batch['A'], dim=1, keepdim=True).to(device)  # [N, 1, H, W]
        
        # 모델 예측
        gray_restored, color_restored = model(inputs)  # 모델 예측

        # 예측된 이미지를 저장
        for i, result in enumerate(color_restored):
            result_img = (result.permute(1, 2, 0).cpu().numpy() * 255).astype(np.uint8)  # [C, H, W] -> [H, W, C]
            output_path = os.path.join(output_dir, f"output_{idx}_{i}.png")
            plt.imsave(output_path, result_img)
            print(f"Saved: {output_path}")